## 아이온 이탈 유저 예측
> 클래스를 분류하기 보다는 실제 회귀식을 구하는 것이 목적이나, 일반 분류기로는 예측하기는 어렵기 때문에 클래스 구분으로 일단 한 번 더 시도해보기로 함

### 1. 클래스 맵핑
> 0 - 5 사이의 값은 0, 6 ~ 15 사이 값은 5, ... , 55+ 이상은 55 으로 클래스 맵핑
> 구글 닥스에 업로드 후 vslookup 통하여 관련 전처리 수행하고 csv 로 다운로드

### 2. 추가 특질
> id, week 정보로 sort 한 이후에 접속 횟수를 특질로 추가
> 맥 로컬에서 sort 후 저장

### 3.  선형 회귀 기울기를 4주차 간 수행하여 특질로 변경
> 정렬된 데이터를 기준으로 세션처리를 통해서 특질(최대접속주차, 최대 4주간의 기울기를 특질 별로 생성)
> linear regression 통해서 각 지표의 4주 간의 X = { 1, 2, 3, 4 } Y = { y1, y2, y3, y4 } 기울기를 특질로 추가

In [165]:
# -- 세션처리 통해서 actor_account_id 당 4주의 데이터를 묶는 작업  -- 25개 필드
#!/usr/bin/env python
# -*- coding:utf -*-
import sys, csv
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# 파일을 읽어서 Dictionary 형태의 List 로 반환하는 함수
def dictionize(filename):
    with open(filename) as f:
        reader = csv.DictReader(f)
        data = [r for r in reader]
    return data

# 파일을 읽어서 List of List 형태로 반환하는 함수
def listinize(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        next(reader) # skip header
        data = [r for r in reader]
    return data

""" 
-- 사전 정의된 (23개)
actor_account_id : 계정 ID
combine_cnt : 제작 횟수
dice_cnt : 주사위 횟수
enter_dd_cnt : 주간 접속 일자 수
exp_get : 경험치 획득량
fortress_cnt : 요새전 보상 횟수
get_ap :AP 획득량
get_gp : GP 획득량
glide_cnt : 활강 횟수
harvest_cnt : 채집 횟수
inc_kina_sum :키나 획득량 (개인간 거래 포함)
indun_cnt : 인스턴스 던전 입장 횟수
kina_sys_inc : 개인간 거래를 제외한 키나 증가량 (e.g. 사냥, 퀘스트 등으로 획득한 키나)
kina_sys_dec : 개인간 거래를 제외한 키나 감소량 (e.g. NPC 구매, 텔레포트 등으로 사용한 키나)
npc_sell_kinasum : 상점 판매로 획득한 키나량
pay_amt_total : 한 주간 구매한 금액 (주별로 표준화 되어있음)
pvp_cnt : PvP 횟수
playtime_ss : 플레이타임(초)
pve_cnt : NPC 킬 횟수
quest_cnt : 퀘스트 완료 횟수
teleport_cnt : 텔레포트 횟수
use_scroll_cnt : 주문서 사용 횟수
week : 플레이한 주 차 ( 계정별 최대 4주 )

-- 추가 지표 증가량 lm.coef_ (13개))

x_combine_cnt
x_enter_dd_cnt
x_exp_get
x_fortress_cnt
x_get_ap
x_get_gp
x_harvest_cnt
x_inc_kina_sum
x_playtime_ss
x_pve_cnt
x_pvp_cnt
x_quest_cnt
x_use_scroll_cnt

-- 정답 label (2개)
z_survival_time : 유지 일자 
z_survival : 유지 그룹 라벨 (c_0, c_5, c_15, c_25, c_35, c_45, c_55) 총 6개 그룹으로 정의

"""

# 입력받은 튜플들의 각 값들의  주어진 킷값들에 대해서 평균치를 저장하여 하나의 dict 로 반환
#
# arguements sessions as list[dict]
# returns dict
#
def average_sessions(sessions, keys_sessions):
    average = sessions[0]
    average["week"] = 0
    average["y_survival"] = average.pop("survival")
    average["y_survival_time"] = average.pop("survival_time")
    
    num_of_sessions = len(sessions)
    if num_of_sessions == 1:
        average["week"] = 1
        return average
    
    for session in sessions:
        for key in keys_sessions:
            value = session.get(key)
            if key == "actor_account_id":
                average[key] = value
            elif key == "week":
                average[key] = int(average.get("week", 0)) + 1
            else:
                average[key] = float(average.get(key, 0.0)) + float(value)
                
    for key in keys_sessions:
        if key != "actor_account_id" and key != "week":
            average[key] = float(average.get(key)) / num_of_sessions
            
    return average

# 입력 받은 킷값에 대하여 주어진 튜플들의 선형 회귀 직선의 기울기를 추가하여 x_${key} 값으로 dict 반환
# 단 튜플 수가 1개이면 모든 값은 0.0으로 반환
def extract_coef(sessions, keys):
    coef = {}
    num_of_sessions = len(sessions)
    # 튜플 수가 1개인 경우만 먼저 적용
    if len(sessions) == 1:
        coef = { "x_" + key:0.0 for key in keys }
    else:
        X = np.asarray(list(range(num_of_sessions)), dtype=np.float)
        X_train = X.reshape(num_of_sessions, 1)
        regressor = LinearRegression()
        for key in keys:
            Y = np.asarray([ sessions[x][key] for x in range(num_of_sessions) ], dtype=np.float)
#             Y = preprocessing.normalize(Y, norm='l2') # normalize 하면 수치가 너무 작아져서 coef 값이 변별력이 떨어짐.
            y_train = Y.reshape(num_of_sessions, 1)
            regressor.fit(X_train, y_train)
            coef["x_" + key] = regressor.coef_[0][0]# 기울기는 coef 이고, 절편이 coef 이다.!
    coef_array = np.asarray(list(coef.values()), dtype=np.float) # dict.values => list => array 후 normalize
    coef_norm = coef_array / np.linalg.norm(coef_array)
    return coef_norm

keys_sessions = [ "actor_account_id", "combine_cnt", "dice_cnt", "enter_dd_cnt", "exp_get"
                 , "fortress_cnt" , "get_ap", "get_gp", "glide_cnt", "harvest_cnt"
                 , "inc_kina_sum" ,  "indun_cnt", "kina_sys_inc", "kina_sys_dec", "npc_sell_kinasum"
                 , "pay_amt_total" , "pvp_cnt", "playtime_ss", "pve_cnt", "quest_cnt"
                 , "teleport_cnt" , "use_scroll_cnt", "week" ] # 23개
keys_coef = [ "combine_cnt", "enter_dd_cnt", "exp_get", "fortress_cnt", "get_ap"
                   , "get_gp", "harvest_cnt", "inc_kina_sum", "playtime_ss", "pve_cnt"
                   , "pvp_cnt", "quest_cnt", "use_scroll_cnt" ] # 13개
keys_labels = [ "survival", "survival_time" ] # 2개

# 최대 4개의 로그를 받아서 최종 출력 대상 튜플을 반환하는 함수
# 적어도 2개 이상의 튜플이 있어야 증감을 측정할 수 있으므로 튜플이 하나이면 모든 값들을 그대로 쓰고, 증감 수치는 0으로 저장
# 일반 필드의 값들은 평균치를 취하여 값으로 저장하도록 한다.
# 기존의 값들은 그대로 사용하는 것이 좋을 것 같고 증감에 유의미한 필드에 대해서만 선정해서 필드를 추가하도록 수정
def sessionize(sessions):
    x_session = average_sessions(sessions, keys_sessions)
    x_coef = extract_coef(sessions, keys_coef)
    x_session.update(x_coef)
    return x_session

# 세션 목록을 디버깅 하기 위한 함수
def debug_sessions(sessions):
    for session in sessions:
        for key in sorted(session.keys()):
            value = session.get(key)
            print(key, value)
        print(session.get("actor_account_id"), session.get("week"), session.get("y_survival_time"))
        print(session)
        
def create_headers():
    headers = keys_sessions + [ "x_" + key for key in keys_coef ] + [ "y_" + key for key in keys_labels ]
    return headers

# 생성된 dict 객체를 파일로 저장하는 함수
def store_sessions(filename, sessions, headers):
    with open(filename, 'w+') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(headers)
        for session in sessions:
           # writer.writerow([ [value] for key, value in sorted(session.items()) ])
           writer.writerow([ value for key, value in sorted(session.items()) ])
            
            
def main(source_data, target_data):
    # 파일을 읽어서 동일한 actor_account_id 값을 하나로 묶고, actor 당, 출석일수, 행동의 coef 값을 반환하는 함수
    items = dictionize(source_data)
    prev_id = None
    curr_id = None
    sessions = []
    x_sessions = []
    for item in items:
        curr_id = item.get("actor_account_id")
        if prev_id != None and prev_id != curr_id:
            x_session = sessionize(sessions)
            x_sessions.append(x_session)
            sessions.clear()
        prev_id = curr_id
        sessions.append(item)

    if len(sessions) > 0:
        x_session = sessionize(sessions)
        x_sessions.append(x_session)

    headers = create_headers()
    store_sessions(target_data, x_sessions, headers)
    


def test_extract_coef(source):
    items = dictionize(source)
    sessions = []
    x_sessions = []
    x_coef = extract_coef(items, keys_coef)
    print(x_coef)
    

__TEST__ = False

if __name__ == "__main__":
    source="data/data_label_sorted.csv"
    target="data/data_label_output.csv"
    
    if __TEST__:
        source="./source.csv"
        target="./target.csv"
        test_extract_coef(source)
    else:
        main(source, target)
        


[ -7.14005906e-27  -3.45134058e-10   1.23385426e-08   4.89820108e-02
  -5.46462259e-09  -3.90116530e-07   2.01328201e-10   9.98799661e-01
   0.00000000e+00   3.65266878e-09   2.99211096e-06  -8.39826208e-09
  -3.80337732e-07]


In [147]:
# from sklearn.linear_model import LinearRegression
X = np.asarray([1, 2, 3, 4], dtype=np.float)
X_train = X.reshape(4,1)
Y = np.asarray([2,3,3,2], dtype=np.float)
Y = preprocessing.normalize(Y, norm='l2')
y_train = Y.reshape(4,1)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
# print(Y, y_train)
# print(regressor.intercept_)
print(regressor.coef_)

[[ -6.20633538e-18]]


In [161]:
x = np.random.rand(1000)*10
print(x.shape)
type(x)

(1000,)


numpy.ndarray

In [30]:
keys_intercepts = [ "exp_get", "pvp_cnt", "quest_cnt", "inc_kina_sum", "enter_dd_cnt", "playtime_ss", "pve_cnt", "get_ap", "get_gp",  "fortress_cnt", "harvest_cnt", "combine_cnt", "use_scroll_cnt" ]
print(len(keys_intercepts))
print(keys_intercepts[1])
data = { "x_" + key:0.0 for key in keys_intercepts }
for key in sorted(data.keys()):
    value = data.get(key)
    print(key, value)


13
pvp_cnt
x_combine_cnt 0.0
x_enter_dd_cnt 0.0
x_exp_get 0.0
x_fortress_cnt 0.0
x_get_ap 0.0
x_get_gp 0.0
x_harvest_cnt 0.0
x_inc_kina_sum 0.0
x_playtime_ss 0.0
x_pve_cnt 0.0
x_pvp_cnt 0.0
x_quest_cnt 0.0
x_use_scroll_cnt 0.0


In [65]:
names = ["a", "b", "c"]
y = [ {"a":"1", "b":"10", "c":"100"}, {"a":"2", "b":"20", "c":"30"}, {"a":"3", "b":"30", "c":"300"} ]
num_of_y = len(y)
import numpy as np
for name in names:
    Y = np.asarray([ y[i][name] for i in range(num_of_y) ], dtype=np.float)
    Y = Y.reshape(num_of_y, 1)
    print(Y)

# Y = np.asarray([1.389234,8.389,19.233,0.8], dtype=np.float)

[[ 1.]
 [ 2.]
 [ 3.]]
[[ 10.]
 [ 20.]
 [ 30.]]
[[ 100.]
 [  30.]
 [ 300.]]


In [86]:
sessions = [ {"x":"20000", "y":"10", "z":"100"}, {"a":"2", "b":"20", "c":"30"}, {"a":"3", "b":"30", "c":"300"} ]
for session in sessions:
    print([[value] for key, value in sorted(session.items())])

[['20000'], ['10'], ['100']]
[['2'], ['20'], ['30']]
[['3'], ['30'], ['300']]
